In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pyvallocation.views import FlexibleViewsProcessor, BlackLittermanProcessor
from pyvallocation.portfolioapi import AssetsDistribution, PortfolioWrapper
from pyvallocation import probabilities, moments
os.chdir('..')

# load daily close data for some ETFs
df = pd.read_csv('examples/ETF_prices.csv',index_col=0,parse_dates=True)
print('Loaded ', df.columns)

# resample to weekly frequency
weekly_prices = df.resample('W').ffill()

# compute compounded returns
weekly_returns = np.log(weekly_prices).diff().dropna()

# store the returns shape
T, N = weekly_returns.shape

# inputs/parameters
ANNUALIZATION_FACTOR = 52
HALF_LIFE = ANNUALIZATION_FACTOR * 10 # 10 years

ModuleNotFoundError: No module named 'pyvallocation'

In [ ]:
# generate uniform probabilities
p_uniform = probabilities.generate_uniform_probabilities(T)
print('Effective number of scenarios of probabilities with uniform dist:', probabilities.compute_effective_number_scenarios(p_uniform))

mu_uniform, cov_uniform = moments.estimate_sample_moments(weekly_returns,p_uniform)
mu_uniform_jorion = moments.shrink_mean_jorion(mu_uniform,cov_uniform,T)
cov_uniform_lw_cc = moments.shrink_covariance_ledoit_wolf(weekly_returns,cov_uniform,target='constant_correlation')

Effective number of scenarios of probabilities with uniform dist: 1006.0000000000003


In [ ]:

# Suppose we think S&P 500 will deliver a <2% annualized return in the future
fv = FlexibleViewsProcessor(
    prior_mean=mu_uniform_jorion,
    prior_cov=cov_uniform_lw_cc,
    mean_views={("SPY"): ("<=", 0.02/ANNUALIZATION_FACTOR)},
)
mu, cov = fv.get_posterior()

# create asset distribution class
ad = AssetsDistribution(mu,cov)

# initialize portfolio wrapper
pw = PortfolioWrapper(ad)
pw.get_minrisk_portfolio()

No custom constraints specified, using default ones


array([0.17035479, 0.10075132, 0.23611218, 0.49278171])

In [4]:
fv.get_posterior_probabilities().sum()

np.float64(0.9999999541401108)